In [1]:
from agents import Agent, function_tool, WebSearchTool, FileSearchTool
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

In [2]:
search_agent = Agent(
    name = "SearchAgent",
    instructions = (
        "You immediately provide an input to the WebSearchTool to find up-to-date information on the user's query."
    ),
    tools = [WebSearchTool()],
)

In [3]:
from openai import OpenAI
import os

client = OpenAI()

def upload_file(file_path: str, vector_store_id: str):
    file_name = os.path.basename(file_path)
    try:
        file_response = client.files.create(file = open(file_path, 'rb'), purpose = "assistants")
        attach_response = client.vector_stores.files.create(
            vector_store_id = vector_store_id,
            file_id = file_response.id
        )
        return {"file": file_name, "status": "success"}
    
    except Exception as e:
        print(f"Error with {file_name}: {str(e)}")
        return {"file": file_name, "status":"failed", "error":str(e)}

#create a vector store which returns a dictionary

def create_vector_store(store_name: str) -> dict:
    try:
        vector_store = client.vector_stores.create(name = store_name)
        details = {
            "id":vector_store.id,
            "name":vector_store.name,
            "created_at":vector_store.created_at,
            "file_count":vector_store.file_counts.completed
        }
        print("Vector store created: ", details)
        return details
    
    except Exception as e:
        print(f"Error creating vector store: {e}")
        return {}
    
vector_store_id = create_vector_store("Elly's beauty Product Knowledge Base")
upload_file("voice_agents_knowledge/acme_product_catalogue.pdf", vector_store_id["id"])


Vector store created:  {'id': 'vs_67e6e378b078819183608dea190ba442', 'name': "Elly's beauty Product Knowledge Base", 'created_at': 1743184760, 'file_count': 0}


{'file': 'acme_product_catalogue.pdf', 'status': 'success'}

In [14]:
knowledge_agent = Agent(
    name = "KnowledgeAgent",
    instructions= (
        "You answer user questions on our product portfolio with concise, helpful responses using the FileSearchtool."   
    ),

    tools = [FileSearchTool(
        max_num_results = 3,
        vector_store_ids= ["vs_67e6e378b078819183608dea190ba442"],
    ),],
)

In [15]:
# --- Tool 1: Fetch account information (dummy) ---

@function_tool
def get_account_info(user_id: str) -> dict:
    """Returns dummy account information for a given user."""
    return{
        "user_id": user_id,
        "name": "Markus Harry",
        "account_balance": "$72.50",
        "membership_status": "Gold Executive"
    }

# --- Agent: Account Agent ---

account_agent = Agent(
    name="AccountAgent",
    instructions = (
        "You provide account information based on a user ID using the get_account_info tool."
    ),
    tools = [get_account_info],
)

In [16]:
# --- Agent: Triage Agent ---

triage_agent = Agent(
    name = "Assistant",
    instructions = prompt_with_handoff_instructions("""

You are the virtual assistant for Elly's Shop. Welcome the user and ask how you can help.
Based on the user's intent, route to:
- AccountAgent for account-related queries
- KnowledgeAgent for product FAQs
- SearchAgent for anything requiring real-time web search
                                                    
"""),
handoffs = [account_agent, knowledge_agent, search_agent],
)

In [17]:
from agents import Runner, trace 

async def test_queries():
    examples = [
        "What's my Elly's account balance doc? My user ID is 9856215499", # Account Agent test
        "Ooh i've got money to spend! How big is the input and how fast is the output of the dynamite dispenser?", # Knowledge Agent test
        "Hmmm, what about duck hunting gear - what's trending right now?", # Search Agent test
    ]

    with trace("Elly's App Assistant"):
        for query in examples:
            result = await Runner.run(triage_agent, query)
            print(f"User: {query}")
            print(result.final_output)
            print("---")
            
await test_queries()

User: What's my Elly's account balance doc? My user ID is 9856215499
Your account balance is $72.50, Markus Harry. You're also a Gold Executive member.
---
User: Ooh i've got money to spend! How big is the input and how fast is the output of the dynamite dispenser?
The Automated Dynamite Dispenser has a capacity of 10 sticks and dispenses them at a speed of 1 stick every 2 seconds.
---
User: Hmmm, what about duck hunting gear - what's trending right now?
Staying updated with the latest trends in duck hunting gear can significantly enhance your hunting experience. Here are some of the top trending items for the 2025 season:



**Banded Aspire Catalyst Waders**  
These all-season waders feature waterproof-breathable technology, minimal-stitch construction for enhanced mobility, and PrimaLoft Aerogel insulation inserts for thermal protection. ([blog.gritroutdoors.com](https://blog.gritroutdoors.com/must-have-duck-hunting-gear-for-a-winning-season/?utm_source=openai))




**Sitka Delta Zip

In [19]:
import numpy as np
import sounddevice as sd 
from agents.voice import AudioInput, SingleAgentVoiceWorkflow, VoicePipeline


In [ ]:
async def voice_assistant():
    samplerate = sd.query_devices(kind = 'input')['default_samplerate']

    while True:
        pipeline = VoicePipeline(workflow = SingleAgentVoiceWorkflow(triage_agent))

        cmd = input("Press Enter to start recording... or esc to exit")
        if cmd.lower() == "esc":
            print("Exiting...")
            break

        print("Listening...")
        recorded_chunks = []

        with sd.InputStream(samplerate = samplerate, channels = 1, dtype = 'int16', callback = lambda indata, frames, time, status: recorded_chunks.append(indata.copy()) ):
            input()

        recording = np.concatenate(recorded_chunks, axis = 0)

        audio_input = AudioInput(buffer = recording)

        with trace("Voice Assistant"):
            result = await pipeline.run(audio_input)


        response_chunks = []
        async for event in result.stream():
            if event.type == "voice_stream_event_audio":
                response_chunks.append(event.data)

        response_audio = np.concatenate(response_chunks, axis = 0)

        print("Assistent is responding...")
        sd.play(response_audio, samplerate = samplerate)
        sd.wait()
        print("---")

await voice_assistant()


AttributeError: module 'sounddevice' has no attribute 'querydevices'